In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import streamlit as st

In [18]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [19]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/administration-overview")
loader

In [20]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='Overview - Docs by LangChainSkip to main content🚀 Share how you\'re building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageResource HierarchyOrganizationsWorkspacesResource tagsUser Management and RBACUsersAPI keysExpiration DatesPersonal Access Tokens (PATs)Service

In [22]:
for i, d in enumerate(docs):
    print(f"Chunk {i+1} length = {len(d.page_content)}\n")


Chunk 1 length = 23783



In [24]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [25]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content="Overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageResource HierarchyOrganizationsWorkspacesResource tagsUser Management and RBACUsersAPI keysExpiration DatesPersonal Access Tokens (PATs)Service 

In [26]:
from langchain_ollama import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="nomic-embed-text")

In [27]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [28]:
vectorstoredb

In [29]:
# Check similarity scores to see how confident the matches are
query = "A user is a person who has access to LangSmith"
results = vectorstoredb.similarity_search_with_relevance_scores(query, k=5)

for i, (doc, score) in enumerate(results):
    print(f"Result {i+1} (score: {score:.4f}):")
    print(doc.page_content[:200])
    print("---")

Result 1 (score: 0.7724):
​User Management and RBAC
​Users
A user is a person who has access to LangSmith. Users can be members of one or more organizations and workspaces within those organizations.
Organization members are m
---
Result 2 (score: 0.5735):
Roles are used to define the set of permissions that a user has within a workspace. There are three built-in system roles that cannot be edited:
---
Result 3 (score: 0.5723):
Workspace Admin has full access to all resources within the workspace.
Workspace Editor has full permissions except for workspace management (adding/removing users, changing roles, configuring service
---
Result 4 (score: 0.5030):
The following table provdies an overview of organization level permissions:
Organization ViewerOrganization UserOrganization AdminView organization configuration✅✅✅View organization roles✅✅✅View organ
---
Result 5 (score: 0.4920):
By default, keys never expire. Once expired, an API key is no longer valid and cannot be reactivated or ha

In [30]:
## Query From a vector db
query="A user is a person who has access to LangSmith"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'\u200bUser Management and RBAC\n\u200bUsers\nA user is a person who has access to LangSmith. Users can be members of one or more organizations and workspaces within those organizations.\nOrganization members are managed in organization settings:\n\nAnd workspace members are managed in workspace settings:'

In [31]:
from langchain_ollama import OllamaLLM
llm=OllamaLLM(model="llama3.2")

In [32]:
## Retrieval Chain, Document chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| OllamaLLM(model='llama3.2')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [33]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"A user is a person who has access to LangSmith",
    "context":[Document(page_content="A user is a person who has access to LangSmith")]
})

'Based solely on the provided context, it can be inferred that the user "LangSmith" refers to an individual with access to the language or linguistic tool, but no additional information about their identity, role, or characteristics can be determined.'

# Retrieval Chain

In [35]:
retriever=vectorstoredb.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [36]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019F0FC78490>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [38]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"A user is a person who has access to LangSmith?"})
response['answer']

'There is no question provided. The context appears to be discussing user management and RBAC (Role-Based Access Control) within a workspace or organization, but there is no specific question being asked.'